# AC215_milestone3_model_training
Yuqing Pan, Sijia (Nancy) Li, Xiaohan Zhao, Jiashu Xu, Ziqing Luo

### Set Up and Install Dependencies

In [1]:
# Connect to drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


We forked the `LLaVA` repository to `github.com/cnut1648/LLaVA` and modified training scripts `train.py` and `train_mem.py` to utilize our `ScienceQA-LLaVA` dataset.

In [3]:
# Clone our LLaVA repository
!git clone https://github.com/cnut1648/LLaVA

Cloning into 'LLaVA'...
remote: Enumerating objects: 1272, done.
remote: Counting objects: 100% (418/418), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 1272 (delta 348), reused 287 (delta 285), pack-reused 854
Receiving objects: 100% (1272/1272), 11.95 MiB | 9.29 MiB/s, done.
Resolving deltas: 100% (794/794), done.


In [4]:
%ls

drive/  LLaVA/  sample_data/


In [6]:
# Install packages
!pip install transformers datasets evaluate
!pip install ninja
!pip install flash-attn --no-build-isolation
!pip install fire

In [7]:
# Enter the LLaVA directory
%cd LLaVA
%pwd

/content/LLaVA


'/content/LLaVA'

In [8]:
# Install dependencies of LLaVA
!pip install -e .

Obtaining file:///content/LLaVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.9/809.9 kB 64.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
%pwd

'/content/LLaVA'

In [12]:
# Create directory to save model checkpoints
%mkdir checkpoints
%cd checkpoints

/content/LLaVA/checkpoints


In [13]:
# Download mm_projector.bin of the pretrained vicuna-7b model
!wget "https://huggingface.co/liuhaotian/llava-pretrain-vicuna-7b-v1.3/resolve/main/mm_projector.bin"

--2023-10-05 20:55:28--  https://huggingface.co/liuhaotian/llava-pretrain-vicuna-7b-v1.3/resolve/main/mm_projector.bin
Resolving huggingface.co (huggingface.co)... 13.33.33.55, 13.33.33.102, 13.33.33.20, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/fd/18/fd18449765dc2ad27737496941d3c70aa1661a600163330f2c3336e46dc50ef8/466c70d865e395e465d5e93a53fb37b8fe1c338f23b650b430d92cd1bdebdf8b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27mm_projector.bin%3B+filename%3D%22mm_projector.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1696798528&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5Njc5ODUyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mZC8xOC9mZDE4NDQ5NzY1ZGMyYWQyNzczNzQ5Njk0MWQzYzcwYWExNjYxYTYwMDE2MzMzMGYyYzMzMzZlNDZkYzUwZWY4LzQ2NmM3MGQ4NjVlMzk1ZTQ2NWQ1

In [15]:
# Login to wandb to monitor training
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [14]:
%cd ..

/content/LLaVA


### Training Model

In [16]:
! deepspeed llava/train/train_mem.py \
    --deepspeed ./scripts/zero2.json \
    --lora_enable True \
    --bits 4 \
    --model_name_or_path lmsys/vicuna-7b-v1.3 \
    --version v1 \
    --data_path "Not Used" \
    --image_folder "Not Used" \
    --vision_tower openai/clip-vit-large-patch14 \
    --pretrain_mm_mlp_adapter ./checkpoints/mm_projector.bin \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --bf16 True \
    --output_dir ./checkpoints/llava-vicuna-7b-v1.3-pretrain-ScienceQA_QCM_LEA-colab \
    --num_train_epochs 1 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --gradient_accumulation_steps 4 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 500 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 512 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb

[2023-10-05 21:13:00,948] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-10-05 21:13:05.294227: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-10-05 21:13:06,422] [WARNING] [runner.py:196:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2023-10-05 21:13:06,422] [INFO] [runner.py:555:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None llava/train/train_mem.py --deepspeed ./scripts/zero2.json --lora_enable True --bits 4 --model_name_or_path lmsys/vicuna-7b-v1.3 --version v1 --data_path ../ScienceQA/data/scienceqa/llava_train_QCM-LEA.json --image_folder ../ScienceQA/data/scienceqa/images/train --vision_tower openai/clip-vit-large-patch14 --pretrain_mm_mlp_adapter ./checkpoints/mm_projector.bin --mm_vision_select_layer -2

In [17]:
# Copy checkpoints from runtime to drive, will be downloaded and upload to GCS
!cp -r checkpoints/llava-vicuna-7b-v1.3-pretrain-ScienceQA_QCM_LEA-colab  /content/drive/MyDrive/AC215Project/checkpoints/llava-vicuna-7b-v1.3-pretrain-ScienceQA_QCM_LEA-colab

### Scratches for Debugging

In [ ]:
# !pip install transformers datasets evaluate

In [ ]:
import torch
print(torch.__version__)

2.0.1+cu118


In [ ]:
import os
import copy
from dataclasses import dataclass, field
import json
import logging
import pathlib
from typing import Dict, Optional, Sequence, List

import torch

import transformers

from llava.constants import IGNORE_INDEX, IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from torch.utils.data import Dataset
from llava.train.llava_trainer import LLaVATrainer

from llava import conversation as conversation_lib
from llava.model import *
from llava.mm_utils import tokenizer_image_token

from PIL import Image


local_rank = None

In [ ]:
import datasets
ds = datasets.load_dataset("cnut1648/ScienceQA-LLAVA")
ds.save_to_disk("ScienceQA-LLAVA")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/12726 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4241 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12726 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4241 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4241 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoConfig, LlamaConfig, TrainingArguments, Trainer, AutoTokenizer, PreTrainedTokenizer
from llava.model.language_model.llava_llama import LlavaLlamaForCausalLM
from dataclasses import dataclass, field
import transformers
import bisect
import warnings
import math
from typing import (
    Generic,
    Iterable,
    Iterator,
    List,
    Optional,
    Sequence,
    Tuple,
    TypeVar,
    Union
)

# No 'default_generator' in torch/__init__.pyi
from torch import default_generator, randperm
from torch._utils import _accumulate
import json

In [ ]:
class LlavaConfig(LlamaConfig):
    model_type = "llava"
AutoConfig.register("llava", LlavaConfig)
# model = LlavaLlamaForCausalLM.from_pretrained("liuhaotian/llava-llama-2-13b-chat-lightning-preview")
model = LlavaLlamaForCausalLM.from_pretrained("liuhaotian/LLaVA-Lightning-MPT-7B-preview")

You are using a model of type llava_mpt to instantiate a model of type llava. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlavaLlamaForCausalLM were not initialized from the model checkpoint at liuhaotian/LLaVA-Lightning-MPT-7B-preview and are newly initialized: ['layers.21.mlp.down_proj.weight', 'layers.0.self_attn.v_proj.weight', 'layers.2.input_layernorm.weight', 'layers.24.mlp.gate_proj.weight', 'layers.12.post_attention_layernorm.weight', 'layers.2.self_attn.o_proj.weight', 'layers.11.mlp.gate_proj.weight', 'layers.2.mlp.up_proj.weight', 'layers.1.post_attention_layernorm.weight', 'layers.6.self_attn.o_proj.weight', 'layers.13.mlp.up_proj.weight', 'layers.0.mlp.up_proj.weight', 'layers.27.mlp.gate_proj.weight', 'layers.0.self_attn.q_proj.weight', 'layers.7.self_attn.k_proj.weight', 'layers.25.post_attention_layernorm.weight', 'layers.17.mlp.up_proj.weight', 'layers.19.mlp.up_proj.weight', 'layers.29.self_attn.k_proj.weight', 'layers.28.self_attn.v_proj.weight', 'norm.weight', 'layers.16.mlp.down_proj.weight', 'layers.17.self_attn.q_proj.weight', 'layers.28.self_attn.rotary_emb.inv_fre

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
            "liuhaotian/LLaVA-Lightning-MPT-7B-preview",
            model_max_length=512,
            padding_side="right",
            use_fast=True,
        )

In [ ]:
training_args = TrainingArguments(
    adafactor=False,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    auto_find_batch_size=False,
    bf16=False,
    bf16_full_eval=False,
    data_seed=None,
    dataloader_drop_last=False,
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    ddp_backend=None,
    ddp_broadcast_buffers=None,
    ddp_bucket_cap_mb=None,
    ddp_find_unused_parameters=None,
    ddp_timeout=1800,
    debug=[],
    deepspeed="./scripts/zero2.json",
    disable_tqdm=False,
    do_eval=False,
    do_predict=False,
    do_train=False,
    eval_accumulation_steps=None,
    eval_delay=0,
    eval_steps=None,
    evaluation_strategy="no",
    fp16=False,
    fp16_backend="auto",
    fp16_full_eval=False,
    fp16_opt_level="O1",
    fsdp=[],
    fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
    fsdp_min_num_params=0,
    fsdp_transformer_layer_cls_to_wrap=None,
    full_determinism=False,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    greater_is_better=None,
    group_by_length=False,
    half_precision_backend="auto",
    hub_model_id=None,
    hub_private_repo=False,
    hub_strategy="every_save",
    hub_token="<HUB_TOKEN>",
    ignore_data_skip=False,
    include_inputs_for_metrics=False,
    jit_mode_eval=False,
    label_names=None,
    label_smoothing_factor=0.0,
    learning_rate=2e-05,
    length_column_name="length",
    load_best_model_at_end=False,
    local_rank=0,
    log_level="passive",
    log_level_replica="warning",
    log_on_each_node=True,
    logging_dir="./checkpoints/llava-vicuna-7b-v1.3-pretrain_lcs558k_plain-ScienceQA_QCM_LEA-12e/runs/Oct03_22-25-04_cc74909a614c",
    logging_first_step=False,
    logging_nan_inf_filter=True,
    logging_steps=1.0,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_grad_norm=1.0,
    max_steps=-1,
    metric_for_best_model=None,
    mp_parameters="",
    no_cuda=False,
    num_train_epochs=1.0,
    optim="adamw_torch",
    optim_args=None,
    output_dir="./checkpoints/llava-vicuna-7b-v1.3-pretrain_lcs558k_plain-ScienceQA_QCM_LEA-12e",
    overwrite_output_dir=False,
    past_index=-1,
    per_device_eval_batch_size=2,
    per_device_train_batch_size=2,
    prediction_loss_only=False,
    push_to_hub=False,
    push_to_hub_model_id=None,
    push_to_hub_organization=None,
    push_to_hub_token="<PUSH_TO_HUB_TOKEN>",
    ray_scope="last",
    remove_unused_columns=False,
    report_to=['wandb'],
    resume_from_checkpoint=None,
    run_name="./checkpoints/llava-vicuna-7b-v1.3-pretrain_lcs558k_plain-ScienceQA_QCM_LEA-12e",
    save_on_each_node=False,
    save_safetensors=False,
    save_steps=50000,
    save_strategy="steps",
    save_total_limit=1,
    seed=42,
    sharded_ddp=[],
    skip_memory_metrics=True,
    tf32=None,
    torch_compile=False,
    torch_compile_backend=None,
    torch_compile_mode=None,
    torchdynamo=None,
    tpu_metrics_debug=False,
    tpu_num_cores=None,
    use_ipex=False,
    use_legacy_prediction_loop=False,
    use_mps_device=False,
    warmup_ratio=0.03,
    warmup_steps=0,
    weight_decay=0.0,
    xpu_backend=None,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1617: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


In [ ]:
@dataclass
class DataArguments:
    data_path: str = field(default=None,
                           metadata={"help": "Path to the training data."})
    lazy_preprocess: bool = False
    is_multimodal: bool = False
    image_folder: str = field(default=None)
    image_aspect_ratio: str = 'square'
    image_grid_pinpoints: str = field(default=None)

data_args = DataArguments(
    # data_path='../ScienceQA/data/scienceqa/llava_train_QCM-LEA.json',
    data_path='../ScienceQA/data/scienceqa/llava_minitest_QCM-LEA.json',
    lazy_preprocess=True,
    is_multimodal=True,
    image_folder='../ScienceQA/data/images/test',
    image_aspect_ratio='square',
    image_grid_pinpoints=None
)

In [ ]:
def preprocess(
    sources: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
    has_image: bool = False
) -> Dict:
    """
    Given a list of sources, each is a conversation list. This transform:
    1. Add signal '### ' at the beginning each sentence, with end signal '\n';
    2. Concatenate conversations together;
    3. Tokenize the concatenated conversation;
    4. Make a deepcopy as the target. Mask human words with IGNORE_INDEX.
    """
    if conversation_lib.default_conversation.sep_style == conversation_lib.SeparatorStyle.PLAIN:
        return preprocess_plain(sources, tokenizer)
    if conversation_lib.default_conversation.sep_style == conversation_lib.SeparatorStyle.LLAMA_2:
        return preprocess_llama_2(sources, tokenizer, has_image=has_image)
    if conversation_lib.default_conversation.version.startswith("v1"):
        return preprocess_v1(sources, tokenizer, has_image=has_image)
    if conversation_lib.default_conversation.version == "mpt":
        return preprocess_mpt(sources, tokenizer)
    # add end signal and concatenate together
    conversations = []
    for source in sources:
        header = f"{conversation_lib.default_conversation.system}\n\n"
        conversation = _add_speaker_and_signal(header, source)
        conversations.append(conversation)
    # tokenize conversations
    def get_tokenize_len(prompts):
        return [len(tokenizer_image_token(prompt, tokenizer)) for prompt in prompts]

    if has_image:
        input_ids = [tokenizer_image_token(prompt, tokenizer, return_tensors='pt') for prompt in conversations]
    else:
        conversations_tokenized = _tokenize_fn(conversations, tokenizer)
        input_ids = conversations_tokenized["input_ids"]

    targets = copy.deepcopy(input_ids)
    for target, source in zip(targets, sources):
        if has_image:
            tokenized_lens = get_tokenize_len([header] + [s["value"] for s in source])
        else:
            tokenized_lens = _tokenize_fn([header] + [s["value"] for s in source], tokenizer)["input_ids_lens"]
        speakers = [sentence["from"] for sentence in source]
        _mask_targets(target, tokenized_lens, speakers)

    return dict(input_ids=input_ids, labels=targets)

In [ ]:
def preprocess_multimodal(
    sources: Sequence[str],
    data_args: DataArguments
) -> Dict:
    is_multimodal = data_args.is_multimodal
    if not is_multimodal:
        return sources

    for source in sources:
        for sentence in source:
            if DEFAULT_IMAGE_TOKEN in sentence['value']:
                sentence['value'] = sentence['value'].replace(DEFAULT_IMAGE_TOKEN, '').strip()
                sentence['value'] = DEFAULT_IMAGE_TOKEN + '\n' + sentence['value']
                sentence['value'] = sentence['value'].strip()
                if "mmtag" in conversation_lib.default_conversation.version:
                    sentence['value'] = sentence['value'].replace(DEFAULT_IMAGE_TOKEN, '<Image>' + DEFAULT_IMAGE_TOKEN + '</Image>')
            replace_token = DEFAULT_IMAGE_TOKEN
            if data_args.mm_use_im_start_end:
                replace_token = DEFAULT_IM_START_TOKEN + replace_token + DEFAULT_IM_END_TOKEN
            sentence["value"] = sentence["value"].replace(DEFAULT_IMAGE_TOKEN, replace_token)

    return sources

In [ ]:
class Dataset:
    """An abstract class representing a :class:`Dataset`.

    All datasets that represent a map from keys to data samples should subclass
    it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
    data sample for a given key. Subclasses could also optionally overwrite
    :meth:`__len__`, which is expected to return the size of the dataset by many
    :class:`~torch.utils.data.Sampler` implementations and the default options
    of :class:`~torch.utils.data.DataLoader`.

    .. note::
      :class:`~torch.utils.data.DataLoader` by default constructs a index
      sampler that yields integral indices.  To make it work with a map-style
      dataset with non-integral indices/keys, a custom sampler must be provided.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

In [ ]:
class LazySupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str,
                 tokenizer: transformers.PreTrainedTokenizer,
                 data_args: DataArguments):
        super(LazySupervisedDataset, self).__init__()
        list_data_dict = json.load(open(data_path, "r"))

        self.tokenizer = tokenizer
        self.list_data_dict = list_data_dict
        self.data_args = data_args

    def __len__(self):
        return len(self.list_data_dict)

    def __getitem__(self, i):
        sources = self.list_data_dict[i]
        if isinstance(i, int):
            sources = [sources]
        assert len(sources) == 1, "Don't know why it is wrapped to a list"  # FIXME
        if 'image' in sources[0]:
            image_file = self.list_data_dict[i]['image']
            image_folder = self.data_args.image_folder
            processor = self.data_args.image_processor
            image = Image.open(os.path.join(image_folder, image_file)).convert('RGB')
            if self.data_args.image_aspect_ratio == 'pad':
                def expand2square(pil_img, background_color):
                    width, height = pil_img.size
                    if width == height:
                        return pil_img
                    elif width > height:
                        result = Image.new(pil_img.mode, (width, width), background_color)
                        result.paste(pil_img, (0, (width - height) // 2))
                        return result
                    else:
                        result = Image.new(pil_img.mode, (height, height), background_color)
                        result.paste(pil_img, ((height - width) // 2, 0))
                        return result
                image = expand2square(image, tuple(int(x*255) for x in processor.image_mean))
                image = processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
            else:
                image = processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
            sources = preprocess_multimodal(
                copy.deepcopy([e["conversations"] for e in sources]),
                self.data_args)
        else:
            sources = copy.deepcopy([e["conversations"] for e in sources])
        data_dict = preprocess(
            sources,
            self.tokenizer,
            has_image=('image' in self.list_data_dict[i]))
        if isinstance(i, int):
            data_dict = dict(input_ids=data_dict["input_ids"][0],
                             labels=data_dict["labels"][0])

        # image exist in the data
        if 'image' in self.list_data_dict[i]:
            data_dict['image'] = image
        elif self.data_args.is_multimodal:
            # image does not exist in the data, but the model is multimodal
            crop_size = self.data_args.image_processor.crop_size
            data_dict['image'] = torch.zeros(3, crop_size['height'], crop_size['width'])
        return data_dict

In [ ]:
@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence):
        input_ids, labels = tuple([instance[key] for instance in instances]
                                  for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids,
            batch_first=True,
            padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels,
                                                 batch_first=True,
                                                 padding_value=IGNORE_INDEX)
        input_ids = input_ids[:, :self.tokenizer.model_max_length]
        labels = labels[:, :self.tokenizer.model_max_length]
        batch = dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

        if 'image' in instances[0]:
            images = [instance['image'] for instance in instances]
            if all(x is not None and x.shape == images[0].shape for x in images):
                batch['images'] = torch.stack(images)
            else:
                batch['images'] = images

        return batch

In [ ]:
def make_supervised_data_module(tokenizer: PreTrainedTokenizer,
                                data_args):
    """Make dataset and collator for supervised fine-tuning."""
    train_dataset = LazySupervisedDataset(tokenizer=tokenizer,
                                data_path=data_args.data_path,
                                data_args=data_args)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    return dict(train_dataset=train_dataset,
                eval_dataset=None,
                data_collator=data_collator)


In [ ]:
data_module = make_supervised_data_module(tokenizer=tokenizer,
                                          data_args=data_args)

In [ ]:
trainer = Trainer(
    model=model,
    **data_module,
    tokenizer=tokenizer,
    args=training_args
)

In [4]:
# https://stackoverflow.com/questions/73747731/runtimeerror-cuda-out-of-memory-how-setting-max-split-size-mb?noredirect=1&lq=1
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [ ]:
trainer.train()

OutOfMemoryError: ignored